In [1]:
## This file implements neural networks and logistic regression on p11presabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 95.5% for combination data, and 93.13% for over-sampling data 
## after improvememt and regularization.
## For logistic regression, the accuracy is 98.2% for combination data, and 91.6% for over-sampling data.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 100% for combination data, and 96.18% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 97.65% for combination data, and 94.43% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p11presabs_quant.csv')
df.shape

(255, 974)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.971625
1      0.591125
2      0.902750
3      1.169875
4      1.147000
5      1.112875
6      0.936500
7      0.707000
8      0.806375
9      0.921500
10     0.510000
11     0.958000
12     1.036750
13     0.427625
14     1.052250
15     1.079500
16     0.390375
17     0.385125
18     1.025500
19     0.225750
20     0.295875
21     0.348250
22     0.507375
23     1.114000
24     1.089625
25     0.163000
26     0.888125
27     0.198375
28     0.828250
29     1.153375
         ...   
225    0.975300
226    0.698300
227    0.933700
228    1.012100
229    1.104800
230    0.621000
231    0.941250
232    0.928500
233    0.902250
234    0.897000
235    1.047375
236    0.992625
237    0.901250
238    1.075750
239    0.758000
240    0.831750
241    0.891000
242    0.369125
243    0.772000
244    0.986750
245    0.975375
246    1.125250
247    0.828750
248    1.086250
249    0.854750
250    0.905000
251    0.572500
252    1.071000
253    1.000500
254    0.955125
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.971625
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.591125
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.902750
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.169875
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.147000


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

1    218
0     37
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 974)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 973)

In [13]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 972) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 201), (1, 166)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 256 samples, validate on 111 samples
Epoch 1/100
256/256 [==============================] - 0s 1ms/step - loss: 0.5434 - accuracy: 0.8047 - val_loss: 0.4208 - val_accuracy: 0.8829
Epoch 2/100
256/256 [==============================] - 0s 76us/step - loss: 0.4199 - accuracy: 0.8555 - val_loss: 0.3606 - val_accuracy: 0.8739
Epoch 3/100
256/256 [==============================] - 0s 82us/step - loss: 0.3857 - accuracy: 0.8711 - val_loss: 0.3416 - val_accuracy: 0.8919
Epoch 4/100
256/256 [==============================] - 0s 86us/step - loss: 0.3786 - accuracy: 0.8789 - val_loss: 0.3039 - val_accuracy: 0.8829
Epoch 5/100
256/256 [==============================] - 0s 87us/step - loss: 0.3437 - accuracy: 0.8789 - val_loss: 0.2703 - val_accuracy: 0.9369
Epoch 6/100
256/256 [==============================] - 0s 85us/step - loss: 0.3124 - accuracy: 0.8945 - val_loss: 0.3416 - val_accuracy: 0.8468
Epoch 7/100
256/256 [==============================] - 0s 76us/step - loss: 0.3291 - accura

256/256 [==============================] - 0s 83us/step - loss: 0.1347 - accuracy: 0.9570 - val_loss: 0.1263 - val_accuracy: 0.9730
Epoch 58/100
256/256 [==============================] - 0s 80us/step - loss: 0.1454 - accuracy: 0.9531 - val_loss: 0.1569 - val_accuracy: 0.9369
Epoch 59/100
256/256 [==============================] - 0s 87us/step - loss: 0.1231 - accuracy: 0.9648 - val_loss: 0.1043 - val_accuracy: 0.9550
Epoch 60/100
256/256 [==============================] - 0s 76us/step - loss: 0.1212 - accuracy: 0.9570 - val_loss: 0.1014 - val_accuracy: 0.9820
Epoch 61/100
256/256 [==============================] - 0s 85us/step - loss: 0.1237 - accuracy: 0.9609 - val_loss: 0.1178 - val_accuracy: 0.9730
Epoch 62/100
256/256 [==============================] - 0s 79us/step - loss: 0.1201 - accuracy: 0.9531 - val_loss: 0.1053 - val_accuracy: 0.9820
Epoch 63/100
256/256 [==============================] - 0s 78us/step - loss: 0.1150 - accuracy: 0.9727 - val_loss: 0.0982 - val_accuracy: 0.964

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

111/111 [==============================] - 0s 137us/step
combination test accuracy: 98.20%


In [25]:
#### add regularizer and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [26]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 256 samples, validate on 111 samples
Epoch 1/100
256/256 [==============================] - 0s 957us/step - loss: 2.1466 - accuracy: 0.4883 - val_loss: 0.7145 - val_accuracy: 0.5586
Epoch 2/100
256/256 [==============================] - 0s 137us/step - loss: 1.7888 - accuracy: 0.5859 - val_loss: 0.6390 - val_accuracy: 0.6847
Epoch 3/100
256/256 [==============================] - 0s 155us/step - loss: 1.8785 - accuracy: 0.6523 - val_loss: 0.6164 - val_accuracy: 0.7027
Epoch 4/100
256/256 [==============================] - 0s 144us/step - loss: 2.2437 - accuracy: 0.7422 - val_loss: 0.5982 - val_accuracy: 0.7117
Epoch 5/100
256/256 [==============================] - 0s 138us/step - loss: 1.6432 - accuracy: 0.7852 - val_loss: 0.5812 - val_accuracy: 0.7297
Epoch 6/100
256/256 [==============================] - 0s 137us/step - loss: 1.6151 - accuracy: 0.8047 - val_loss: 0.5693 - val_accuracy: 0.7207
Epoch 7/100
256/256 [==============================] - 0s 138us/step - loss: 1.8420 

256/256 [==============================] - 0s 102us/step - loss: 2.1825 - accuracy: 0.8125 - val_loss: 0.3449 - val_accuracy: 0.9459
Epoch 57/100
256/256 [==============================] - 0s 89us/step - loss: 1.6896 - accuracy: 0.8594 - val_loss: 0.3378 - val_accuracy: 0.9369
Epoch 58/100
256/256 [==============================] - 0s 80us/step - loss: 1.4171 - accuracy: 0.8594 - val_loss: 0.3526 - val_accuracy: 0.9369
Epoch 59/100
256/256 [==============================] - 0s 93us/step - loss: 1.2714 - accuracy: 0.8906 - val_loss: 0.3239 - val_accuracy: 0.9550
Epoch 60/100
256/256 [==============================] - 0s 83us/step - loss: 1.2177 - accuracy: 0.8867 - val_loss: 0.3267 - val_accuracy: 0.9459
Epoch 61/100
256/256 [==============================] - 0s 83us/step - loss: 1.6254 - accuracy: 0.8555 - val_loss: 0.3160 - val_accuracy: 0.9640
Epoch 62/100
256/256 [==============================] - 0s 80us/step - loss: 1.8007 - accuracy: 0.8477 - val_loss: 0.3391 - val_accuracy: 0.95

In [28]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

111/111 [==============================] - 0s 83us/step
combination test accuracy: 95.50%


In [29]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [30]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/100
305/305 [==============================] - 0s 452us/step - loss: 0.6764 - accuracy: 0.5672 - val_loss: 0.5943 - val_accuracy: 0.6260
Epoch 2/100
305/305 [==============================] - 0s 94us/step - loss: 0.5753 - accuracy: 0.6689 - val_loss: 0.5381 - val_accuracy: 0.7099
Epoch 3/100
305/305 [==============================] - 0s 196us/step - loss: 0.5297 - accuracy: 0.7803 - val_loss: 0.4938 - val_accuracy: 0.8092
Epoch 4/100
305/305 [==============================] - 0s 149us/step - loss: 0.4949 - accuracy: 0.8066 - val_loss: 0.4763 - val_accuracy: 0.7405
Epoch 5/100
305/305 [==============================] - 0s 116us/step - loss: 0.4775 - accuracy: 0.8066 - val_loss: 0.4993 - val_accuracy: 0.8092
Epoch 6/100
305/305 [==============================] - 0s 113us/step - loss: 0.4555 - accuracy: 0.8197 - val_loss: 0.4371 - val_accuracy: 0.8244
Epoch 7/100
305/305 [==============================] - 0s 101us/step - loss: 0.4456 -

305/305 [==============================] - 0s 87us/step - loss: 0.2642 - accuracy: 0.9082 - val_loss: 0.2996 - val_accuracy: 0.8779
Epoch 58/100
305/305 [==============================] - 0s 84us/step - loss: 0.2724 - accuracy: 0.9082 - val_loss: 0.3002 - val_accuracy: 0.8779
Epoch 59/100
305/305 [==============================] - 0s 84us/step - loss: 0.2650 - accuracy: 0.8951 - val_loss: 0.3145 - val_accuracy: 0.8626
Epoch 60/100
305/305 [==============================] - 0s 86us/step - loss: 0.2580 - accuracy: 0.9082 - val_loss: 0.3563 - val_accuracy: 0.8473
Epoch 61/100
305/305 [==============================] - 0s 82us/step - loss: 0.2558 - accuracy: 0.9115 - val_loss: 0.2945 - val_accuracy: 0.8702
Epoch 62/100
305/305 [==============================] - 0s 84us/step - loss: 0.2700 - accuracy: 0.9016 - val_loss: 0.3331 - val_accuracy: 0.8626
Epoch 63/100
305/305 [==============================] - 0s 89us/step - loss: 0.2637 - accuracy: 0.9049 - val_loss: 0.3547 - val_accuracy: 0.847

In [32]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

131/131 [==============================] - 0s 69us/step
over-sampling test accuracy: 91.60%


In [33]:
#### add regularizer and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [34]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/100
305/305 [==============================] - 0s 1ms/step - loss: 2.4789 - accuracy: 0.4984 - val_loss: 0.7578 - val_accuracy: 0.7786
Epoch 2/100
305/305 [==============================] - 0s 101us/step - loss: 2.6174 - accuracy: 0.5311 - val_loss: 0.6886 - val_accuracy: 0.5954
Epoch 3/100
305/305 [==============================] - 0s 84us/step - loss: 1.9542 - accuracy: 0.7082 - val_loss: 0.6652 - val_accuracy: 0.6336
Epoch 4/100
305/305 [==============================] - 0s 83us/step - loss: 2.2375 - accuracy: 0.7115 - val_loss: 0.6486 - val_accuracy: 0.6718
Epoch 5/100
305/305 [==============================] - 0s 78us/step - loss: 1.7212 - accuracy: 0.7377 - val_loss: 0.6378 - val_accuracy: 0.7099
Epoch 6/100
305/305 [==============================] - 0s 82us/step - loss: 1.6819 - accuracy: 0.7541 - val_loss: 0.6248 - val_accuracy: 0.7710
Epoch 7/100
305/305 [==============================] - 0s 83us/step - loss: 1.8588 - accur

305/305 [==============================] - 0s 88us/step - loss: 1.9439 - accuracy: 0.7967 - val_loss: 0.4846 - val_accuracy: 0.8702
Epoch 58/100
305/305 [==============================] - 0s 83us/step - loss: 1.8227 - accuracy: 0.8262 - val_loss: 0.4535 - val_accuracy: 0.9084
Epoch 59/100
305/305 [==============================] - 0s 110us/step - loss: 2.1874 - accuracy: 0.8000 - val_loss: 0.4690 - val_accuracy: 0.8779
Epoch 60/100
305/305 [==============================] - 0s 93us/step - loss: 2.1769 - accuracy: 0.8000 - val_loss: 0.4749 - val_accuracy: 0.8702
Epoch 61/100
305/305 [==============================] - 0s 81us/step - loss: 1.6731 - accuracy: 0.8328 - val_loss: 0.4591 - val_accuracy: 0.8779
Epoch 62/100
305/305 [==============================] - 0s 81us/step - loss: 1.6325 - accuracy: 0.8164 - val_loss: 0.5035 - val_accuracy: 0.8550
Epoch 63/100
305/305 [==============================] - 0s 80us/step - loss: 1.3911 - accuracy: 0.8361 - val_loss: 0.4719 - val_accuracy: 0.86

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 78us/step
over-sampling test accuracy: 88.55%


In [37]:
#### improve neural network on over-sampling data
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],),activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [38]:
model1_over3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 599us/step - loss: 2.8831 - accuracy: 0.4492 - val_loss: 0.7641 - val_accuracy: 0.7786
Epoch 2/1000
305/305 [==============================] - 0s 85us/step - loss: 2.0220 - accuracy: 0.5279 - val_loss: 0.7002 - val_accuracy: 0.8550
Epoch 3/1000
305/305 [==============================] - 0s 91us/step - loss: 2.3500 - accuracy: 0.4787 - val_loss: 0.6813 - val_accuracy: 0.8473
Epoch 4/1000
305/305 [==============================] - 0s 92us/step - loss: 1.8411 - accuracy: 0.5180 - val_loss: 0.6672 - val_accuracy: 0.8550
Epoch 5/1000
305/305 [==============================] - 0s 113us/step - loss: 2.4294 - accuracy: 0.4787 - val_loss: 0.6679 - val_accuracy: 0.8092
Epoch 6/1000
305/305 [==============================] - 0s 134us/step - loss: 2.0467 - accuracy: 0.6492 - val_loss: 0.6656 - val_accuracy: 0.7939
Epoch 7/1000
305/305 [==============================] - 0s 118us/step - loss: 1.9

305/305 [==============================] - 0s 91us/step - loss: 1.8083 - accuracy: 0.8033 - val_loss: 0.4897 - val_accuracy: 0.8015
Epoch 57/1000
305/305 [==============================] - 0s 101us/step - loss: 2.1034 - accuracy: 0.7902 - val_loss: 0.4206 - val_accuracy: 0.9084
Epoch 58/1000
305/305 [==============================] - 0s 88us/step - loss: 1.4526 - accuracy: 0.8328 - val_loss: 0.4160 - val_accuracy: 0.9084
Epoch 59/1000
305/305 [==============================] - 0s 85us/step - loss: 1.5950 - accuracy: 0.8328 - val_loss: 0.4141 - val_accuracy: 0.9084
Epoch 60/1000
305/305 [==============================] - 0s 97us/step - loss: 1.6008 - accuracy: 0.8197 - val_loss: 0.4122 - val_accuracy: 0.9084
Epoch 61/1000
305/305 [==============================] - 0s 86us/step - loss: 2.0308 - accuracy: 0.7574 - val_loss: 0.4352 - val_accuracy: 0.9008
Epoch 62/1000
305/305 [==============================] - 0s 92us/step - loss: 1.7767 - accuracy: 0.8295 - val_loss: 0.4279 - val_accuracy

305/305 [==============================] - 0s 93us/step - loss: 2.0190 - accuracy: 0.7836 - val_loss: 0.4677 - val_accuracy: 0.8855
Epoch 113/1000
305/305 [==============================] - 0s 85us/step - loss: 1.5883 - accuracy: 0.8033 - val_loss: 0.5434 - val_accuracy: 0.7557
Epoch 114/1000
305/305 [==============================] - 0s 93us/step - loss: 2.0049 - accuracy: 0.7967 - val_loss: 0.4496 - val_accuracy: 0.8779
Epoch 115/1000
305/305 [==============================] - 0s 93us/step - loss: 1.7611 - accuracy: 0.8131 - val_loss: 0.4772 - val_accuracy: 0.8626
Epoch 116/1000
305/305 [==============================] - 0s 99us/step - loss: 2.0393 - accuracy: 0.8000 - val_loss: 0.5103 - val_accuracy: 0.8092
Epoch 117/1000
305/305 [==============================] - 0s 89us/step - loss: 2.0442 - accuracy: 0.7869 - val_loss: 0.4216 - val_accuracy: 0.8855
Epoch 118/1000
305/305 [==============================] - 0s 89us/step - loss: 1.8959 - accuracy: 0.7410 - val_loss: 0.5361 - val_acc

Epoch 168/1000
305/305 [==============================] - 0s 87us/step - loss: 1.8588 - accuracy: 0.8164 - val_loss: 0.3580 - val_accuracy: 0.9084
Epoch 169/1000
305/305 [==============================] - 0s 84us/step - loss: 1.7302 - accuracy: 0.8393 - val_loss: 0.3470 - val_accuracy: 0.9084
Epoch 170/1000
305/305 [==============================] - 0s 85us/step - loss: 1.9237 - accuracy: 0.8393 - val_loss: 0.3691 - val_accuracy: 0.9008
Epoch 171/1000
305/305 [==============================] - 0s 88us/step - loss: 1.5375 - accuracy: 0.8262 - val_loss: 0.3715 - val_accuracy: 0.9084
Epoch 172/1000
305/305 [==============================] - 0s 99us/step - loss: 1.4875 - accuracy: 0.8459 - val_loss: 0.4346 - val_accuracy: 0.7710
Epoch 173/1000
305/305 [==============================] - 0s 91us/step - loss: 1.9385 - accuracy: 0.7836 - val_loss: 0.4662 - val_accuracy: 0.6794
Epoch 174/1000
305/305 [==============================] - 0s 93us/step - loss: 1.9196 - accuracy: 0.8131 - val_loss: 0

Epoch 224/1000
305/305 [==============================] - 0s 95us/step - loss: 1.4689 - accuracy: 0.8623 - val_loss: 0.4333 - val_accuracy: 0.8779
Epoch 225/1000
305/305 [==============================] - 0s 93us/step - loss: 1.4339 - accuracy: 0.8557 - val_loss: 0.4179 - val_accuracy: 0.8931
Epoch 226/1000
305/305 [==============================] - 0s 96us/step - loss: 1.5474 - accuracy: 0.8721 - val_loss: 0.3910 - val_accuracy: 0.9160
Epoch 227/1000
305/305 [==============================] - 0s 84us/step - loss: 1.5152 - accuracy: 0.8590 - val_loss: 0.3968 - val_accuracy: 0.8779
Epoch 228/1000
305/305 [==============================] - 0s 84us/step - loss: 1.7394 - accuracy: 0.8328 - val_loss: 0.3924 - val_accuracy: 0.8855
Epoch 229/1000
305/305 [==============================] - 0s 85us/step - loss: 1.8218 - accuracy: 0.8295 - val_loss: 0.3912 - val_accuracy: 0.8855
Epoch 230/1000
305/305 [==============================] - 0s 85us/step - loss: 2.2486 - accuracy: 0.8098 - val_loss: 0

...

305/305 [==============================] - 0s 93us/step - loss: 1.7151 - accuracy: 0.8459 - val_loss: 0.3141 - val_accuracy: 0.9084
Epoch 724/1000
305/305 [==============================] - 0s 88us/step - loss: 1.6664 - accuracy: 0.8459 - val_loss: 0.3242 - val_accuracy: 0.9008
Epoch 725/1000
305/305 [==============================] - 0s 87us/step - loss: 1.8220 - accuracy: 0.8361 - val_loss: 0.3074 - val_accuracy: 0.9237
Epoch 726/1000
305/305 [==============================] - 0s 88us/step - loss: 1.7635 - accuracy: 0.8426 - val_loss: 0.3073 - val_accuracy: 0.9084
Epoch 727/1000
305/305 [==============================] - 0s 95us/step - loss: 2.0638 - accuracy: 0.8230 - val_loss: 0.3267 - val_accuracy: 0.9008
Epoch 728/1000
305/305 [==============================] - 0s 90us/step - loss: 1.8719 - accuracy: 0.8328 - val_loss: 0.3063 - val_accuracy: 0.9237
Epoch 729/1000
305/305 [==============================] - 0s 86us/step - loss: 1.9638 - accuracy: 0.8295 - val_loss: 0.3184 - val_acc

Epoch 779/1000
305/305 [==============================] - 0s 89us/step - loss: 1.4706 - accuracy: 0.8590 - val_loss: 0.3029 - val_accuracy: 0.9084
Epoch 780/1000
305/305 [==============================] - 0s 88us/step - loss: 1.7081 - accuracy: 0.8492 - val_loss: 0.3099 - val_accuracy: 0.9084
Epoch 781/1000
305/305 [==============================] - 0s 86us/step - loss: 1.3432 - accuracy: 0.8787 - val_loss: 0.3084 - val_accuracy: 0.9084
Epoch 782/1000
305/305 [==============================] - 0s 86us/step - loss: 2.0656 - accuracy: 0.8230 - val_loss: 0.2951 - val_accuracy: 0.9237
Epoch 783/1000
305/305 [==============================] - 0s 88us/step - loss: 1.8937 - accuracy: 0.8426 - val_loss: 0.2997 - val_accuracy: 0.9237
Epoch 784/1000
305/305 [==============================] - 0s 96us/step - loss: 1.7656 - accuracy: 0.8426 - val_loss: 0.3055 - val_accuracy: 0.9237
Epoch 785/1000
305/305 [==============================] - 0s 94us/step - loss: 1.6929 - accuracy: 0.8525 - val_loss: 0

305/305 [==============================] - 0s 89us/step - loss: 1.6563 - accuracy: 0.8525 - val_loss: 0.2915 - val_accuracy: 0.9237
Epoch 835/1000
305/305 [==============================] - 0s 98us/step - loss: 1.8457 - accuracy: 0.8426 - val_loss: 0.3037 - val_accuracy: 0.9237
Epoch 836/1000
305/305 [==============================] - 0s 96us/step - loss: 2.0784 - accuracy: 0.8131 - val_loss: 0.3049 - val_accuracy: 0.9237
Epoch 837/1000
305/305 [==============================] - 0s 91us/step - loss: 1.7033 - accuracy: 0.8492 - val_loss: 0.3077 - val_accuracy: 0.9237
Epoch 838/1000
305/305 [==============================] - 0s 93us/step - loss: 1.3068 - accuracy: 0.8754 - val_loss: 0.3146 - val_accuracy: 0.9160
Epoch 839/1000
305/305 [==============================] - 0s 93us/step - loss: 1.4538 - accuracy: 0.8656 - val_loss: 0.2972 - val_accuracy: 0.9466
Epoch 840/1000
305/305 [==============================] - 0s 87us/step - loss: 1.9654 - accuracy: 0.8295 - val_loss: 0.3048 - val_acc

Epoch 890/1000
305/305 [==============================] - 0s 93us/step - loss: 1.9022 - accuracy: 0.8393 - val_loss: 0.3189 - val_accuracy: 0.9084
Epoch 891/1000
305/305 [==============================] - 0s 87us/step - loss: 1.6313 - accuracy: 0.8623 - val_loss: 0.2984 - val_accuracy: 0.9084
Epoch 892/1000
305/305 [==============================] - 0s 86us/step - loss: 1.4005 - accuracy: 0.8689 - val_loss: 0.3035 - val_accuracy: 0.9084
Epoch 893/1000
305/305 [==============================] - 0s 87us/step - loss: 1.3900 - accuracy: 0.8754 - val_loss: 0.2847 - val_accuracy: 0.9389
Epoch 894/1000
305/305 [==============================] - 0s 93us/step - loss: 1.9428 - accuracy: 0.8393 - val_loss: 0.2997 - val_accuracy: 0.9237
Epoch 895/1000
305/305 [==============================] - 0s 88us/step - loss: 1.3877 - accuracy: 0.8754 - val_loss: 0.2830 - val_accuracy: 0.9466
Epoch 896/1000
305/305 [==============================] - 0s 93us/step - loss: 1.3212 - accuracy: 0.8852 - val_loss: 0

Epoch 946/1000
305/305 [==============================] - 0s 89us/step - loss: 2.0242 - accuracy: 0.8393 - val_loss: 0.2975 - val_accuracy: 0.9313
Epoch 947/1000
305/305 [==============================] - 0s 88us/step - loss: 1.4604 - accuracy: 0.8623 - val_loss: 0.2816 - val_accuracy: 0.9466
Epoch 948/1000
305/305 [==============================] - 0s 87us/step - loss: 1.4502 - accuracy: 0.8656 - val_loss: 0.3017 - val_accuracy: 0.9237
Epoch 949/1000
305/305 [==============================] - 0s 89us/step - loss: 1.8565 - accuracy: 0.8393 - val_loss: 0.2942 - val_accuracy: 0.9313
Epoch 950/1000
305/305 [==============================] - 0s 89us/step - loss: 1.9640 - accuracy: 0.8262 - val_loss: 0.3028 - val_accuracy: 0.9237
Epoch 951/1000
305/305 [==============================] - 0s 88us/step - loss: 1.8048 - accuracy: 0.8393 - val_loss: 0.2984 - val_accuracy: 0.9313
Epoch 952/1000
305/305 [==============================] - 0s 85us/step - loss: 1.7203 - accuracy: 0.8426 - val_loss: 0

In [40]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

131/131 [==============================] - 0s 86us/step
over-sampling test accuracy: 93.13%


In [41]:
############ Logistic Regression ############

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [43]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.20%


In [45]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 91.60%


In [47]:
############## Random Forest ##############

In [48]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [49]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [50]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [51]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 96.18%


In [52]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [53]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98076923 0.96078431 0.98039216 1.         0.96078431]
0.9765460030165911


In [54]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.93548387 0.98387097 0.86885246 0.96666667 0.96666667]
0.9443081262118808
